In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

## Dataset



In [ ]:
import pandas as pd
df = pd.read_csv("/content/Banking.csv")

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df.head(10)

,question,answer,text
4571,What are the different types of credit cards o...,Banks offer a range of credit cards such as tr...,##Human:\ngenerate a easy to understand output...
1255,How can I apply for a loan to expand my techno...,To apply for a technology startup expansion lo...,##Human:\ngenerate a easy to understand output...
3800,How can I apply for a loan under the Pradhan M...,"To apply for a loan under PM-SYM in India, app...",##Human:\ngenerate a easy to understand output...
4163,How do banks in India handle fixed deposit (FD...,Banks in India allow FD premature withdrawals ...,##Human:\ngenerate a easy to understand output...
3786,How can I apply for a loan under the Pradhan M...,"To apply for a loan under PMMY in India, appro...",##Human:\ngenerate a easy to understand output...
1018,What is the process for updating the address l...,To update the address linked to your Aadhaar c...,##Human:\ngenerate a easy to understand output...
1436,How can I apply for a loan to expand my eco-fr...,To apply for an eco-friendly products business...,##Human:\ngenerate a easy to understand output...
180,How can I check my account balance using SMS b...,To check your account balance using SMS bankin...,##Human:\ngenerate a easy to understand output...
4827,What are the features and benefits of a bank's...,Senior citizen fixed deposit schemes offer hig...,##Human:\ngenerate a easy to understand output...
432,How can I apply for a loan to purchase agricul...,To apply for a loan to buy agricultural equipm...,##Human:\ngenerate a easy to understand output...


In [ ]:
test_df.head(10)

,question,answer,text
290,What is the role of the Financial Stability an...,The Financial Stability and Development Counci...,##Human:\ngenerate a easy to understand output...
4051,What is the process for availing a loan under ...,Individuals can avail PMRY loans in India by a...,##Human:\ngenerate a easy to understand output...
1665,How can I apply for a loan to expand my e-lear...,To apply for an e-learning startup expansion l...,##Human:\ngenerate a easy to understand output...
4060,What are the benefits of using the Unified Pay...,UPI enables quick and secure digital payments ...,##Human:\ngenerate a easy to understand output...
1044,What is the process for updating the mobile nu...,To update the mobile number linked to your PAN...,##Human:\ngenerate a easy to understand output...
2304,What is the role of the National Payments Corp...,NPCI plays a critical role in the Indian payme...,##Human:\ngenerate a easy to understand output...
4640,What are the advantages of using digital payme...,Digital payment methods like UPI and mobile wa...,##Human:\ngenerate a easy to understand output...
3869,How can I protect my bank account from unautho...,"To protect your bank account in India, use str...",##Human:\ngenerate a easy to understand output...
2654,How can I apply for a loan to start a small-sc...,To apply for a loan to start a small-scale man...,##Human:\ngenerate a easy to understand output...
4369,What is the process for applying for a busines...,Business loan applicants need to provide busin...,##Human:\ngenerate a easy to understand output...


In [ ]:
from datasets import Dataset,DatasetDict
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

## Loading the model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "guardrail/llama-2-7b-guanaco-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/1.96G [00:00<?, ?B/s]

...

model-00012-of-00014.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 500
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [ ]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/3894 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.954600
20,1.172200
30,0.887100
40,0.716300
50,0.652100
60,0.654800
70,0.603700
80,0.528700
90,0.563100
100,0.553200


TrainOutput(global_step=500, training_loss=0.5178762407302856, metrics={'train_runtime': 2606.0127, 'train_samples_per_second': 3.07, 'train_steps_per_second': 0.192, 'total_flos': 2.7009276141797376e+16, 'train_loss': 0.5178762407302856, 'epoch': 2.05})

During training, the model should converge nicely as follows:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model.

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [ ]:
df['text']

0       ##Human:\ngenerate a easy to understand output...
1       ##Human:\ngenerate a easy to understand output...
2       ##Human:\ngenerate a easy to understand output...
3       ##Human:\ngenerate a easy to understand output...
4       ##Human:\ngenerate a easy to understand output...
                              ...                        
4863    ##Human:\ngenerate a easy to understand output...
4864    ##Human:\ngenerate a easy to understand output...
4865    ##Human:\ngenerate a easy to understand output...
4866    ##Human:\ngenerate a easy to understand output...
4867    ##Human:\ngenerate a easy to understand output...
Name: text, Length: 4868, dtype: object

In [ ]:
text = "How can I apply for a credit card with my bank?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

How can I apply for a credit card with my bank?
 everybody has a credit score, but not everyone knows what it is or how to improve it. Here are some tips to help you improve your credit score:
1. Check your credit report: Obtain a copy of your credit report from each of the three major credit reporting agencies (Equifax, Experian, and TransUnion) and review it for errors or inaccuracies. Dispute any errors you find and work to resolve them.
2. Pay your bills on time: Payment history is a significant factor in determining your credit score, so make sure to pay all of your bills on time, every time. Set up payment reminders or automate your payments to help you stay on track.
3. Keep your credit utilization ratio low: Your credit utilization ratio is the amount of credit you're using compared to the amount of credit available to you. Keeping this ratio low (less than 30%)


In [ ]:
from huggingface_hub import login
login()

In [ ]:
model.push_to_hub("Give name to your repository")

ValueError: Token is required (write-access action) but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.